In [52]:
import matplotlib.pyplot as plt
import numpy as np

MTOW = 15
BATT_WEIGHT = 3

# CONSTANTS
AIR_DEN = 1.25
GRAVITY = 9.82

def calculate_score(takeoff_dist,
                    max_speed, 
                    min_speed, 
                    flight_distance,
                    battery_capacity,
                    payload_mass,
                    payload_volume,
                    landing_angle=5, 
                    landing_dist=10,
                    climb_angle=20, 
                    glide_ratio=3, ):

    payload_energy_consumed = (battery_capacity/(4.2*6))/(flight_distance*payload_mass)

    reference_airplane = np.array([
        30,		# Take-off Distance
        12,		# Climb Angle
        3,		# Glide Ratio
        30,		# Max Speed
        10,		# Min Speed
        5,		# Landing Angle
        50,		# Landing Distance
        15,		# Flight Distance
        0.4,	# Specific Energy Consumtion
        5,		# Payload Mass
        0.5		# Payload Volume
	])
    
    our_plane = np.array([
        takeoff_dist,		# Take-off Distance
        climb_angle,		# Climb Angle
        glide_ratio,		# Glide Ratio
        max_speed,		# Max Speed
        min_speed,		# Min Speed
        landing_angle,		# Landing Angle
        landing_dist,		# Landing Distance
        flight_distance,		# Flight Distance
        payload_energy_consumed,	# Specific Energy Consumtion
    	payload_mass,		# Payload Mass
        payload_volume		# Payload Volume
	])
    
    weights = np.array([
        -4,
        7,
        25,
        4,
        -4,
        5,
        -3,
        8,
        -12,
        10,
        18
	])
    div = our_plane/reference_airplane
    scores = weights*np.tanh(div - 1)
    score = np.sum(scores)

    #print("Div:", div)
    #print("Scores: ", scores)
    
    print("Take off Distance: ", scores[0])
    print("Climb angle: ", scores[1])
    print("Glide Ratio ", scores[2])
    print("Max Speed: ", scores[3])
    print("Min Speed: ", scores[4])
    print("Landing Angle: ", scores[5])
    print("Landing Distance: ", scores[6])
    print("Flight Distanace: ", scores[7])
    print("Specific Energy Consumtion: ", scores[8])
    print("Payload Mass: ", scores[9])
    print("Payload Volume: ", scores[10])

    #print(score)
    

In [30]:
def take_off(max_power, max_thrust, wing_area, fusilage_area, fusilage_cd, take_off_cd, take_off_cl):
    stall_velocity = (2*MTOW*GRAVITY/(take_off_cl*AIR_DEN*wing_area))**0.5

    #takeoff_drag = 0.5*takeoff_cd*AIR_DEN*(fusilage_area + wing_area)*takeoff_velocity**2
    #takeoff_accel = (max_thrust - takeoff_drag)/MTOW

    dt = 0.0001
    current_velocity = 0
    current_time = 0
    current_energy = 0
    current_distance = 0

    takeoff_accel = np.array(())
    takeoff_velocity = np.array(())
    x = np.array(())

    while current_velocity < stall_velocity:
        current_drag = 0.5*take_off_cd*AIR_DEN*(fusilage_area + wing_area)*current_velocity**2
        current_accel = (max_thrust - current_drag)/MTOW
        
        current_distance = current_distance + current_velocity*dt + 0.5*current_accel*dt**2
        current_velocity = current_velocity + current_accel*dt
        
        current_energy = current_energy + max_power*dt
        
        current_time = current_time+dt
        
        #takeoff_accel = np.append(takeoff_accel, current_accel)
        #takeoff_velocity = np.append(takeoff_velocity, current_velocity)
        #x = np.append(x, current_time)
        
    takeoff_time = current_time
    takeoff_energy = current_energy/3600
    takeoff_distance = current_distance

    print(f"Takeoff time: {round(takeoff_time,1)} s")
    print(f"Energy: {round(takeoff_energy,1)} Wh")
    print(f"Takeoff distance: {round(takeoff_distance, 1)} m")

    return takeoff_distance, takeoff_energy

In [ ]:
def climb():
    drag = 0.5*cruise_cd*AIR_DEN*(fusilage_area + wing_area)*cruise_airspeed**2
    lift = 0.5*cruise_cl*AIR_DEN*(wing_area)*cruise_airspeed**2

In [33]:
def endurance(cruise_speed, energy_left, wing_area, fusilage_area, cruise_cd):
    area_drag = wing_area + fusilage_area
    cruise_drag = 0.5*AIR_DEN*cruise_cd*(area_drag)*cruise_speed**2
    print(f"Cruise Drag: {round(cruise_drag)}")
    power = float(input("Input power required: "))

    cruise_time = energy_left/power
    cruise_distance = cruise_speed*cruise_time

    print(f"Cruise Time: {round(cruise_time, 1)}")
    print(f"Cruise Distance: {round(cruise_distance, 1)}")
    return cruise_distance


In [ ]:
# Energy storage in Wh, assuming 260 Wh/kg and 3 kg of batteries
battery_energy = 260*3

max_thrust = 50
power = 600

landing_gear_weight = 0.95
motor_weight = 0.15
prop_weight = 0.05

main_body = 5
fligtcontroller = 0.07

payload = 15 - BATT_WEIGHT - landing_gear_weight - motor_weight*2 - prop_weight*2 - main_body - fligtcontroller
payload_vol = 0.5
# Needs a better estimation of Cd_fuse!!! Scaled with fuselage surface area
fusilage_cd = 0.02

#----- Optim
cruise_airspeed = 15 # m/s
max_power = 600 # Watt
wing_area = 2 # m^2
fusilage_area = 1 # m^2

diameter = 22 # inches
pitch = 12 # inches
p_d = diameter/pitch

take_off_cd = 0.13
take_off_cl = 1.3

cruise_cd = 0.01

stall_velocity = (2*MTOW*GRAVITY/(take_off_cl*AIR_DEN*wing_area))**0.5


take_off_dist, take_off_energy = take_off( 
    max_thrust=max_thrust,
    max_power=max_power,
    wing_area=wing_area,
    fusilage_area=fusilage_area,
    fusilage_cd=fusilage_cd,
    take_off_cd=take_off_cd,
    take_off_cl=take_off_cl
    )

endurance_energy = battery_energy - take_off_energy

cruise_distance = endurance(energy_left=endurance_energy, wing_area=wing_area, fusilage_area=fusilage_area, cruise_cd=cruise_cd, cruise_speed=cruise_airspeed)
print("=====================")
print("SCORES")

calculate_score(takeoff_dist=take_off_dist,
                max_speed=30,
                min_speed=stall_velocity,
                flight_distance=cruise_distance,
                battery_capacity=battery_energy,
                payload_mass=payload,
                payload_volume=payload_vol)

Takeoff time: 3.4 s
Energy: 0.6 Wh
Takeoff distance: 17.9 m
Cruise Drag: 4
Cruise Time: 1.9
Cruise Distance: 29.2
=====================0
SCORES
Take off Distance:  1.5260197071036932
Climb angle:  4.079480617435371
Glide Ratio  0.0
Max Speed:  0.0
Min Speed:  0.1915224910222708
Landing Angle:  0.0
Landing Distance:  1.9921103108035472
Flight Distanace:  5.913063316573077
Specific Energy Consumtion:  5.783633939413468
Payload Mass:  1.1548248661898017
Payload Volume:  0.0
